## Loading the dependencies

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments,Trainer, pipeline
from datasets import load_dataset
import numpy as np
import evaluate
from sklearn.metrics import classification_report
from bertviz import model_view,head_view
import shap
from collections import Counter, defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import os

from nltk import ne_chunk, pos_tag, word_tokenize
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')

### Clarify and reflect on the definition of the term "fake news", which may vary among databases, sometimes non-binary.
Fake news has many definitions, ranging from being factually incorrect to misleading, which makes it hard when quantifying results and cross-examining results between different studies, since the latent space of a fitted model can be very different dependent on the definition of the task. The problem with using objective truth as the definition, that truth can vary depending on culture and context. An actor in a conflict can be seen as the good freedom fighter fighting an oppressive regime by one side, and as a terrorist on the other side. Lastly, the truth can also change over time, which means that models have to be retrained with up-to-date information constantly to be able to combat the fake news within the category factual correct. A conception of fake news based on 'clickbait' (as used in [1]) would be an inherently easier task since the model would not have to have as complex a world model in order to classify correctly. It should be solvable by using information only present in the text and comparing that to the title. 


### Research, where the data comes from and inspect the data: what are the labels, sources, and authors?
The data used for this project is a fake news dataset which can be found on huggingface under that path: [GonzaloA/fake_news](https://huggingface.co/datasets/GonzaloA/fake_news). The data is described as a "mix of other datasets which are the same scope, the Fake News". Unfullfilled with this rather vague description, we sought to find additional information regarding the data and found this kaggle dataset: [ISOT Fake News Dataset](https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets). We tested for similarity and found that **95%** of our training data titles are identical to data found in the kaggle dataset. An interesting outcome of the kaggle data is that all fake news articles come from **websites** flagged by Politifact and/or Wikipedia and not individual articles, and the true articles come from Reuters. This has the effect that de facto task our model is being trained for is whether or not an article is published by Reuters and not the intended fake news detection.

In [ ]:
fake_data = pd.read_csv("Kaggle/Fake.csv")
true_data = pd.read_csv("Kaggle/True.csv")

our_data = load_dataset('GonzaloA/fake_news')

In [ ]:
kaggle_data = pd.DataFrame(pd.concat([fake_data['title'],true_data['title']],axis = 0).reset_index(drop = True))

In [ ]:
train = pd.DataFrame(our_data['train'])

In [ ]:
df = pd.merge(train['title'],kaggle_data['title'],how = 'outer',indicator = True).drop_duplicates()

In [ ]:
df[df['_merge'] == 'both']['title'].count()/train['title'].count()

### Study the literature on how others approach this task. Check the related literature and select your model architecture of choice: LSTM, ...
[Fake news detection based on news content and social contexts: a transformer-based approach](https://link.springer.com/article/10.1007/s41060-021-00302-z)

The paper [1], written by Shaina Raza & Chen Ding, uses META's BART language model trained on two data sets: NELA-GT-19, which are news articles sourced from multiple sites, and Fakeddit, which is a multimodal dataset from Reddit, consisting of both images and text. The datasets used had more than a binary score, it included labels such as mixed, which is when there is a disagrement whether something is true or false, and categories such as satire into a single category Fake. They discuss their approach of continuously updating the model's training data to retrain the model and stay on top of relevant news. They also assert that freezing a model's weights can quickly make the model outdated since they don't generalize well to future events. Finally, they report an accuracy of 74.89%.

[1] = https://link.springer.com/article/10.1007/s41060-021-00302-z

## Model Training

#### Downloading the base model and getting the tokenizer
The code below is not necessary to run, as we fine-tuned the model and uploaded it to HuggingFace, therefore just go down to the Model inspection part.

In [ ]:
# model_name = "google-bert/bert-base-uncased"

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

#### Loading the data set

In [ ]:
data = load_dataset('GonzaloA/fake_news')

In [ ]:
data = data.remove_columns(['Unnamed: 0','title'])

In [ ]:
with open('predictions/predictions.txt','r') as f:
    predictions = f.read().split('\n')

predictions = [int(i) for i in predictions]

In [ ]:
# tokenized_data = data.map(tokenize_function, batched=True)

#### Sampling from the data set

In [ ]:
# small_train_dataset = tokenized_data["train"].shuffle(seed=42).select(range(100))
# small_eval_dataset = tokenized_data["validation"].shuffle(seed=42).select(range(100))
# small_test_dataset = tokenized_data['test']

#### Outputting the training arguments

In [ ]:
# training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

#### Loading the evaluation metrics

In [ ]:
# metric = evaluate.load("accuracy")

In [ ]:
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

#### Initializing the Trainer object and fine-tuning the model

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_train_dataset,
#     eval_dataset=small_eval_dataset,
#     compute_metrics=compute_metrics,
# )

In [ ]:
# trainer.train()

#### Getting the predictions

In [ ]:
# test_labels = small_test_dataset['label']
# small_test_dataset = small_test_dataset.remove_columns(['label','token_type_ids'])

In [ ]:
# predictions = trainer.predict(small_test_dataset)

In [ ]:
# predicted_labels = predictions.predictions.argmax(axis=1)

#### Saving the model

In [ ]:
# trainer.save_model('bert-base-uncased-fake-news-classification')

In [ ]:
# tokenizer.save_pretrained('bert-base-uncased-fake-news-classification')

#### Saving the predictions

In [ ]:
# with open('predictions/predictions.txt', 'w') as f:
#     for line in predicted_labels:
#         f.write(f"{line}\n")

## Model Inspection

#### Loading the model

In [ ]:
model_name = 'FlorianMi/bert-base-uncased-fake-news-classification'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,output_attentions = True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

#### Evaluating the predictions

In [ ]:
print(classification_report(data['test']['label'],predictions))

In [ ]:
# count = 0
# for i,j in zip(predictions,data['test']['label']):
#     if i != j:
#         print(count)
#     count += 1

#### Exploratory Data Analysis

In [ ]:
count_dict = dict()
for i in data:
    true_news = []
    fake_news = []
    for j in data[i]['label']:
        if j == 0:
            fake_news.append(j)
        else:
            true_news.append(j)
    count_dict[i] = (len(fake_news),len(true_news))

In [ ]:
count_df = pd.DataFrame.from_dict(count_dict,orient = 'index')
count_df.rename(columns = {0:'Fake News',1:'True News'},inplace = True)
count_df.plot.bar(figsize = (10,5));

In [ ]:
len_count_dict = dict()
for i in data:
    true_news = []
    fake_news = []
    for j in data[i]:
        if j['label'] == 0:
            fake_news.append(len(j['text']))
        else:
            true_news.append(len(j['text']))
    len_count_dict[i] = (np.mean(fake_news),np.mean(true_news))

In [ ]:
len_count_df = pd.DataFrame.from_dict(len_count_dict,orient = 'index')
len_count_df.rename(columns = {0:'Fake News',1:'True News'},inplace = True)
len_count_df.plot.bar(figsize = (10,5));

In [ ]:
# tokenizer.get_vocab()

#### Inspecting over-represented tokens and named entities in our corpus

In [ ]:
def compute_ratios(first: list, other: list) -> dict:
    '''
    returns the frequency for each element in first divided by its frequency in other
        - meant to show which elements are overrepresented in first compared to other
    '''
    first_counts = Counter(first)
    other_counts = Counter(other)

    ratios = {}

    for item in first_counts:
        if item in other_counts:
            ratios[item] = first_counts[item] / other_counts[item]
        else:
            # to avoid division by zero, pretends items absent in other occur once instead
            ratios[item] = first_counts[item] 

    return ratios

In [ ]:
# splits up data by label
fake = []
true = []
for article in data['train']:
    if article['label'] == 1: # label == true
        true.append(article)
    else:# label == fake
        fake.append(article)


# merges and tokenizes the text of the fake/non-fake news
true_combined = ' '.join([article['text'] for article in true])
fake_combined = ' '.join([article['text'] for article in fake])
true_tokens = word_tokenize(true_combined)
fake_tokens = word_tokenize(fake_combined)

# computes token frequency in fake news / frequency in non-fake news (and vice versa) and sorts by ratio
true_ratios = compute_ratios(true_tokens, fake_tokens)
fake_ratios = compute_ratios(fake_tokens, true_tokens)
true_ratios_sorted = sorted(list(true_ratios.items()), key=lambda x: x[1], reverse=True)
fake_ratios_sorted = sorted(list(fake_ratios.items()), key=lambda x: x[1], reverse=True)

In [ ]:
# prepares token ratios for plotting

# only plots top 10 highest ratios
n_tokens = 10

x_true = [token for token, _  in true_ratios_sorted[:n_tokens]]
x_fake = [token for token, _  in fake_ratios_sorted[:n_tokens]]
x_true.reverse()
x_fake.reverse()

y_true = [ratio for _, ratio in true_ratios_sorted[:n_tokens]]
y_fake = [ratio for _, ratio in fake_ratios_sorted[:n_tokens]]
y_true.reverse()
y_fake.reverse()

In [ ]:
# samples article to save compute time on the following
size = 400
fake_sample = np.random.choice(fake, size=size)
true_sample = np.random.choice(true, size=size)

# does NER on non-fake news, storing all named entities in list
true_NE = []
true_combined = ' '.join([article['text'] for article in true_sample])
tags = pos_tag(word_tokenize(true_combined))
chunks = ne_chunk(tags)
n_chunks = len(chunks)
for i, chunk in enumerate(chunks):
    if hasattr(chunk, 'label'):
        true_NE.append(' '.join([word[0] for word in chunk]))

# does NER on fake news, storing all named entities in list
fake_NE = []
fake_combined = ' '.join([article['text'] for article in fake_sample])
tags = pos_tag(word_tokenize(fake_combined))
chunks = ne_chunk(tags)
n_chunks = len(chunks)
for i, chunk in enumerate(chunks):
    if hasattr(chunk, 'label'):
        fake_NE.append(' '.join([word[0] for word in chunk]))


# computes named entity frequency in fake news / frequency in non-fake news (and vice versa) and sorts by ratio
true_ratios_NE = compute_ratios(true_NE, fake_NE)
fake_ratios_NE = compute_ratios(fake_NE, true_NE)
true_ratios_sorted_NE = sorted(list(true_ratios_NE.items()), key=lambda x: x[1], reverse=True)
fake_ratios_sorted_NE = sorted(list(fake_ratios_NE.items()), key=lambda x: x[1], reverse=True)

In [ ]:
# prepares named entity ratios for plotting

# only plots top 10 highest ratios
n_entities = 10

x_true_NE = [entities for entities, _  in true_ratios_sorted_NE[:n_entities]]
x_fake_NE = [entities for entities, _  in fake_ratios_sorted_NE[:n_entities]]
x_true_NE.reverse()
x_fake_NE.reverse()

y_true_NE = [ratio for _, ratio in true_ratios_sorted_NE[:n_entities]]
y_fake_NE = [ratio for _, ratio in fake_ratios_sorted_NE[:n_entities]]
y_true_NE.reverse()
y_fake_NE.reverse()

In [ ]:
# plots all the ratios
plt.style.use('dark_background')
fig, axs = plt.subplots(figsize=(12,4), nrows=2, ncols=2)

axs = axs.flatten()

axs[0].barh(x_true, y_true)
axs[0].set_title('Tokens overrepresented in non-fake news')
axs[0].tick_params('y', labelsize=9)

axs[1].barh(x_fake, y_fake)
axs[1].set_title('Tokens overrepresented in fake news')
axs[1].tick_params('y', labelsize=9)

axs[2].barh(x_true_NE, y_true_NE)
axs[2].set_title('Named Entities overrepresented in non-fake news')
axs[2].tick_params('y', labelsize=9)

axs[3].barh(x_fake_NE, y_fake_NE)
axs[3].set_title('Named Entities overrepresented in fake news')
axs[3].tick_params('y', labelsize=9)

fig.tight_layout()
fig.savefig('images/overrepresented.png', dpi=300)
fig.show()

In [ ]:
np.random.seed(100)

print(true[np.random.randint(1, len(true))]['text'])
print()
print(true[np.random.randint(1, len(true))]['text'])
print()
print(true[np.random.randint(1, len(true))]['text'])
print()
print(true[np.random.randint(1, len(true))]['text'])
print()
print(true[np.random.randint(1, len(true))]['text'])

In [ ]:
np.random.seed(100)

print(fake[np.random.randint(1, len(fake))]['text'])
print()
print(fake[np.random.randint(1, len(fake))]['text'])
print()
print(fake[np.random.randint(1, len(fake))]['text'])
print()
print(fake[np.random.randint(1, len(fake))]['text'])
print()
print(fake[np.random.randint(1, len(fake))]['text'])

#### Inspecting the model and the head

In [ ]:
inputs = tokenizer.encode(data['test'][9]['text'], return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [ ]:
# head_view(attention, tokens)

In [ ]:
# model_view(attention, tokens)

In [ ]:
# for i, article in enumerate(data['test']):
#     if len(article['text']) < 500 and article['label'] == 0:
#         print(i)
#         print(article['text'])
#         print()

#### Inspecting the model's attention
Label 0 is Fake | Label 1 is True

In [ ]:
pipe = pipeline('text-classification',model=model_name, top_k=None)

In [ ]:
explainer = shap.Explainer(pipe)

In [ ]:
shap_values_true = explainer([data['test'][15]['text']])

In [ ]:
shap.plots.text(shap_values_true)

In [ ]:
shap_values_fake = explainer([data['test'][274]['text']])

In [ ]:
shap.plots.text(shap_values_fake)